# Text CNN Tryout

In [22]:
import pandas as pd
import numpy as np

pd.options.display.max_colwidth = 100000000
training_data = pd.read_csv("C:\\Users\\ricardo\\Github\\Kaggle\\1910_TMU_EnglishReviewClassification\\Data\\train_data.csv")

In [23]:
data_neg = training_data.iloc[0:12001, 0]
data_pos = training_data.iloc[12001:, 0]

data_neg.to_csv("./data_neg.csv", header=False, index=False)
data_pos.to_csv("./data_pos.csv", header=False, index=False)

# Tryout

In [67]:
import sys
import os
from text_processing_util import TextProcessing
from text_cnn import kimCNN
import _pickle as cPickle

In [68]:
MAX_SEQUENCE_LENGTH = 1000
MAX_NB_WORDS = 10000
EMBEDDING_DIM = 300
VALIDATION_SPLIT = 0.2

In [85]:
pos_file = 'new_rt-polarity.pos'
neg_file = 'new_rt-polarity.neg'
#fname = sys.argv[1]

In [99]:
# Prepare text samples and their labels
print('Processing text dataset')

labels = []
texts = []
with open(pos_file, "rb") as f:
    for line in f:
        labels.append('pos')
        texts.append(line.strip())


with open(neg_file, "rb") as f:
    for line in f:
        labels.append('neg')
        texts.append(line.strip())


print("Found %s texts" %len(texts))
print("Found %s labels" %len(labels))

Processing text dataset
Found 25000 texts
Found 25000 labels


In [100]:
texts = [str(x[1]) for x in texts]
fname = "./word2vec.c"

tp = TextProcessing(texts, labels, EMBEDDING_DIM, MAX_SEQUENCE_LENGTH, MAX_NB_WORDS, VALIDATION_SPLIT)

x_train, y_train, x_val, y_val, word_index = tp.preprocess()
embeddings_index = tp.build_embedding_index_from_word2vec(fname, word_index)
print('Found %s word vectors.' % len(embeddings_index))

Found 84 unique tokens.
Shape of data tensor: (25000, 1000)
Shape of label tensor: (25000, 2)
Indexing word vectors.


ValueError: invalid literal for int() with base 10: b'//'

In [31]:
cPickle.dump([word_index, embeddings_index], open('tokenization_and_embedding.p', 'wb'))

labels_index = tp.labels_index

model = kimCNN(EMBEDDING_DIM, MAX_SEQUENCE_LENGTH, MAX_NB_WORDS, embeddings_index, word_index, labels_index=labels_index)
print(model.summary())

model.fit(x=x_train, y=y_train, batch_size=50, epochs=25 , validation_data=(x_val, y_val))

TypeError: a bytes-like object is required, not 'dict'